<a href="https://colab.research.google.com/github/Aastha031295/FineTuning/blob/main/FineTune_BERT_Sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pandas

In [7]:
!pip install transformers torch datasets scikit-learn pyarrow==14.0.1

INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently

In [1]:
import pandas as pd

In [2]:
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Provide the correct path to your file
# Make sure this path is accurate after mounting your drive
file_path = '/content/drive/MyDrive/combined_dataset.csv'  # Adjust 'MyDrive' if necessary

# Read the CSV file
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
data.head(100)

,text,label
0,"Oh great, just what I needed to make my day be...",negative
1,"Oh fantastic, exactly what I wanted! Getting s...",negative
2,"Oh great, just what I needed to make my day be...",negative
3,"Oh joy, at least it didn't get worse! Searchin...",positive
4,"Oh fantastic, exactly what I wanted! Dealing w...",negative
...,...,...
95,"Oh fantastic, just my luck, right? Trying to o...",negative
96,Could this day get any worse? Finding out your...,negative
97,"I'm thrilled, that's a relief. Running out of ...",negative
98,"Could have been worse, I guess. Getting a rece...",positive


In [5]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# Split the data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['text'], data['label'], test_size=0.2, random_state=42
)

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric

In [9]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Convert the labels to integers (e.g., positive = 1, negative = 0)
label_map = {'positive': 1, 'negative': 0}
# Convert labels to lowercase to match the label_map
train_labels = [label_map[label.lower()] for label in train_labels]
test_labels = [label_map[label.lower()] for label in test_labels]

# Create Dataset objects
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [10]:
# 6. Define the evaluation metric (accuracy)
metric = load_metric("accuracy")

<ipython-input-10-1cd21a466c27>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = torch.Tensor(logits)  # Convert logits to a PyTorch tensor
    predictions = torch.argmax(logits, dim=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load the pre-trained BERT model for binary classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # evaluate at each epoch
    per_device_train_batch_size=8,  # batch size for training
    per_device_eval_batch_size=8,   # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# 8. Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.360300,0.328345
2,0.260300,0.529258
3,0.129500,0.599861


TrainOutput(global_step=7833, training_loss=0.2607794763727479, metrics={'train_runtime': 1882.9588, 'train_samples_per_second': 33.28, 'train_steps_per_second': 4.16, 'total_flos': 4121897793269760.0, 'train_loss': 0.2607794763727479, 'epoch': 3.0})

In [14]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.5998610854148865, 'eval_runtime': 37.1884, 'eval_samples_per_second': 140.42, 'eval_steps_per_second': 17.559, 'epoch': 3.0}


In [15]:
model.save_pretrained("/content/drive/MyDrive/folders/1RiIytRWBSTSF7CzltY9pg5ykXN8dzJc4?usp=drive_link/finetuned_bert_sarcasm_model")
tokenizer.save_pretrained("/content/drive/MyDrive/folders/1RiIytRWBSTSF7CzltY9pg5ykXN8dzJc4?usp=drive_link/finetuned_bert_sarcasm_tokenizer")

('/content/drive/MyDrive/folders/1RiIytRWBSTSF7CzltY9pg5ykXN8dzJc4?usp=drive_link/finetuned_bert_sarcasm_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/folders/1RiIytRWBSTSF7CzltY9pg5ykXN8dzJc4?usp=drive_link/finetuned_bert_sarcasm_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/folders/1RiIytRWBSTSF7CzltY9pg5ykXN8dzJc4?usp=drive_link/finetuned_bert_sarcasm_tokenizer/vocab.txt',
 '/content/drive/MyDrive/folders/1RiIytRWBSTSF7CzltY9pg5ykXN8dzJc4?usp=drive_link/finetuned_bert_sarcasm_tokenizer/added_tokens.json')